Installing the beautiful soup library for web scraping

In [39]:
pip install beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


 Importing the required library for web scraping and sending data request to fetch the required table

In [40]:
import pandas as pd
import numpy as np
import requests
import bs4 as bs
from bs4 import BeautifulSoup
import requests

html_doc = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(html_doc, 'html.parser')
#soup.prettify()

Creating a table out of the scraped data

In [41]:
table = soup.find("table")
output_rows = []
for table_row in table.findAll('tr'):
    columns = table_row.findAll('td')
    output_row = []
    for column in columns:
        output_row.append(column.text)
    output_rows.append(output_row)
#output_rows
    

Applying the conditions asked for in the problem statement

In [42]:
names = ['PostalCode','Borough','Neighbourhood']
df = pd.DataFrame(data=output_rows,columns = names)
df = df.replace('\n','', regex=True)
df = df.dropna(how='all', axis=0)
df['Neighbourhood'] = np.where((df['Neighbourhood'] == 'Not assigned'),df['Borough'],df['Neighbourhood'])
df = df[(df.Borough != 'Not assigned')]
df= df.groupby(['PostalCode','Borough']).agg(lambda x: ','.join(x))
df = df.reset_index()

Reading the shape of final table dataframe

In [46]:
print('The dataframe shape is:',df.shape)
df

The dataframe shape is: (103, 3)


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village,Martin Grove Gardens,Richvie..."
101,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."


Extracting the Latitude and Longitude information

In [47]:
latlong = pd.read_csv('http://cocl.us/Geospatial_data')
latlong.rename(columns={"Postal Code": "PostalCode"})
latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merging the two tables to get the latlong informaion in main table

In [ ]:
df = pd.merge(df,latlong,on=['PostalCode'])